In [1]:
import numpy as np
import metrics_custom

### Custom distance metrics for 3rd order tensors
#### Norm-based distance metrics
Suppose $\mathbf{X} \in \mathbb{R}^{l \times m \times n}$ and $\mathbf{Y} \in \mathbb{R}^{l \times m \times n}$ are the two tensors, we define the distance metric between them, induced by a general norm, as

\begin{eqnarray}
d(\mathbf{X}, \mathbf{Y}) &=& \|\mathbf{X} - \mathbf{Y}\|_{p,q,r} \\
&=& \left( \sum_{i=1}^l \left( \sum_{j=1}^m \left( \sum_{k=1}^n |X_{ijk} \,-\, Y_{ijk}|^r \right)^{\frac{q}{r}} \right)^{\frac{p}{q}} \right)^{\frac{1}{p}}
\end{eqnarray}

This can be seen as a direct extension of the entrywise matrix norms defined, for example, [here](https://en.wikipedia.org/wiki/Matrix_norm#L2,1_and_Lp,q_norms). The norm is defined by the parameters $p, q, r$, which should be integers greater than $0$. The following three special cases of the distance metric should be useful in practice for high dimensional data:  
- $p = q = r = 1$: Similar to the Manhattan distance between vectors.
- $p = q = r = 2$: Similar to the Euclidean distance between vectors or Frobenius norm distance between matrices.
- $p = 1, q = r = 2$: In this case, the distance is the sum of Frobenius norm distances between the matrix slices along the second and third dimensions.

#### Cosine angular distance
This is a distance measure between two tensors derived from the cosine similarity. Note that this may not be a distance metric in the true sense because the triangle inequality may not be satisfied. Recall that for two vector inputs $\mathbf{a}$ and $\mathbf{b}$, suppose $S_{\cos}(\mathbf{a}, \mathbf{b})$ is their cosine similarity in the range $[-1, 1]$, the angular distance between them is defined as $\,d_a(\mathbf{a}, \mathbf{b}) = \frac{1}{\pi} \,\arccos(S_{\cos}(\mathbf{a}, \mathbf{b}))$, which has range $[0, 1]$. 

The cosine similarity between two matrices $\mathbf{A}$ and $\mathbf{B}$ of compatible dimensions is defined as

\begin{eqnarray}
S_{\cos}(\mathbf{A}, \mathbf{B}) &=& \frac{<\mathbf{A}, \mathbf{B}>}{\|\mathbf{A}\|_F \,\|\mathbf{B}\|_F} \\ 
&=& \frac{tr(\mathbf{A}^T \,\mathbf{B})}{\sqrt{tr(\mathbf{A}^T \,\mathbf{A}) tr(\mathbf{B}^T \,\mathbf{B})}}.
\end{eqnarray}

In the case of tensors, we first calculate the average cosine similarity between the individual matrix slices (along the second and third dimension) as follows,

$$
S_{\cos}(\mathbf{X}, \mathbf{Y}) ~=~ \frac{1}{l} \sum_{i=1}^l \frac{tr(\mathbf{X_i}^T \,\mathbf{Y_i})}{\sqrt{tr(\mathbf{X_i}^T \,\mathbf{X_i}) tr(\mathbf{Y_i}^T \,\mathbf{Y_i})}},
$$  
where $\mathbf{X_i}$ and $\mathbf{Y_i}$ are the matrix slices of size $m \times n$. The angular cosine distance between the tensors is then defined as

$$d_a(\mathbf{X}, \mathbf{Y}) ~=~ \frac{1}{\pi} \,\arccos(S_{\cos}(\mathbf{X}, \mathbf{Y}))$$.


In [2]:
# Shape of the tensors. Specified as an integer instead of a tuple to avoid errors related to `numba`
shape = (2, 3, 4)

# xt = 2 * np.ones(shape)
xt = np.random.randn(*shape)
# Flatten into a vector before calling the distance function
x = xt.reshape(-1)

# yt = np.ones(shape)
yt = np.random.randn(*shape)
y = yt.reshape(-1)

# The norm parameters `p, q, r` are specified as a tuple to the keyword argument `norm_type`. 
# For example, `p = q = r = 2` is specified as `norm_type=(2, 2, 2)`.
d = metrics_custom.distance_norm_3tensors(x, y, shape=shape, norm_type=(1, 1, 1))
print("p = q = r = 1: distance = {:f}".format(d))

d = metrics_custom.distance_norm_3tensors(x, y, shape=shape, norm_type=(2, 2, 2))
print("p = q = r = 2: distance = {:f}".format(d))

d = metrics_custom.distance_norm_3tensors(x, y, shape=shape, norm_type=(1, 2, 2))
print("p = 1, q = r = 2: distance = {:f}".format(d))

d = metrics_custom.distance_angular_3tensors(x, y, shape=shape)
print("Cosine angular distance = {:f}".format(d))

p = q = r = 1: distance = 17.018815
p = q = r = 2: distance = 4.787602
p = 1, q = r = 2: distance = 6.750589
Cosine angular distance = 0.443325


### Special cases of the norm-based distance metrics
It can be shown that when $p = \infty$ and $q, r < \infty$, the distance metric reduces to

$$
\|\mathbf{X} - \mathbf{Y}\|_{\infty,q,r} = \left(\max_{i = 1, \cdots, l} \sum_{j=1}^m \left( \sum_{k=1}^n |X_{ijk} \,-\, Y_{ijk}|^r \right)^{\frac{q}{r}} \right)^{\frac{1}{q}}
$$

The following two special cases will be useful in practice:

$$
\|\mathbf{X} - \mathbf{Y}\|_{\infty,2,2} = \sqrt{\max_{i = 1, \cdots, l} \sum_{j=1}^m \sum_{k=1}^n (X_{ijk} \,-\, Y_{ijk})^2}
$$

$$
\|\mathbf{X} - \mathbf{Y}\|_{\infty,1,1} = \max_{i = 1, \cdots, l} \sum_{j=1}^m \sum_{k=1}^n |X_{ijk} \,-\, Y_{ijk}|
$$

In [3]:
# The special value `p = -1` is used to specify the infinite case
d = metrics_custom.distance_norm_3tensors(x, y, shape=shape, norm_type=(-1, 2, 2))
print("p = infty, q = 2, r = 2: distance = {:f}".format(d))

d = metrics_custom.distance_norm_3tensors(x, y, shape=shape, norm_type=(-1, 1, 1))
print("p = infty, q = 1, r = 1: distance = {:f}".format(d))

p = infty, q = 2, r = 2: distance = 3.635977
p = infty, q = 1, r = 1: distance = 9.792169
